## DOCUMENT

In [27]:
!pip install faiss-cpu

In [1]:
import gdown
# Output file name and Google Drive file ID
output = 'Nuclear.zip'
file_id = '1QeYz4v_CNfRF6x8cyowrj7FW9UfLazTm'

# The file is downloaded from Google Drive
gdown.download(id=file_id, output=output, quiet=False)
print('DONE')

import zipfile

zip_file = 'Nuclear.zip'

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall('./extracted_articles')

print('ZIP was opened succesfully.')

Downloading...
From: https://drive.google.com/uc?id=1QeYz4v_CNfRF6x8cyowrj7FW9UfLazTm
To: /content/Nuclear.zip
100%|██████████| 22.6k/22.6k [00:00<00:00, 6.90MB/s]

DONE
ZIP was opened succesfully.


In [2]:
import os
import zipfile
import urllib.parse

# folder name to create in Colab
extract_folder = '/content/extracted_articles/New folder'

# Create if the folder does not exist
if not os.path.exists(extract_folder):
    os.makedirs(extract_folder)

# Name of the downloaded ZIP file
zip_file = '/content/Nuclear.zip'

# Open ZIP file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

# Title and article lists
titles = []
articles = []

print('Reading titles...')

i = 0

# Navigating the extracted file folder
for filename in os.listdir(extract_folder):
    if not filename.endswith('.txt'):
        continue

    file_path = os.path.join(extract_folder, filename)

    with open(file_path, 'rb') as f:
        title = urllib.parse.unquote(filename[:-4])
        title = title.replace('_', ' ')

        if len(title) == 0 or len(title.strip()) == 0:
            print('Empty title for', filename)
            continue

        titles.append(title)
        articles.append(f.read().decode('utf-8'))  # changed to 'utf-8'
        i += 1

        if i % 500 == 0:
            print('Processed {:,}'.format(i))

print('DONE.\n')
print('There are {:,} articles.'.format(len(articles)))


Reading titles...
DONE.

There are 3 articles.


In [3]:
titles[0:4]

['wiki boiling water reactor History of nuclear power',
 'wiki boiling water reactor Horizon Nuclear Power',
 'wiki boiling water reactor Light-water reactor']

In [4]:
articles[0]

'This is a history of nuclear power as realized through the first artificial fission of atoms that would lead to the Manhattan Project and, eventually, to using nuclear fission to generate electricity.  \n\n\n== Origins ==\n\nIn 1932, physicists John Cockcroft, Ernest Walton, and Ernest Rutherford discovered that when lithium atoms were "split" by protons from a proton accelerator, immense amounts of energy were released in accordance with the principle of mass–energy equivalence. However, they and other nuclear physics pioneers Niels Bohr and Albert Einstein believed harnessing the power of the atom for practical purposes anytime in the near future was unlikely. The same year, Rutherford\'s doctoral student James Chadwick discovered the neutron. Experiments bombarding materials with neutrons led Frédéric and Irène Joliot-Curie to discover induced radioactivity in 1934, which allowed the creation of radium-like elements. Further work by Enrico Fermi in the 1930s focused on using slow n

## CHUNK ARTICLES


In [5]:
print('Before splitting, {:,} articles.\n'.format(len(titles)))

passage_titles = []
passages = []

print('Splitting...')

for i in range(len(titles)):
    title = titles[i]
    article = articles[i]

    if len(article) == 0:
        print('Empty article for', title)
        continue

    words = article.split()

    for j in range(0, len(words), 100):
        chunk_words = words[j:j+100]
        chunk = " ".join(chunk_words).strip()

        if len(chunk) == 0:
            continue

        passage_titles.append(title)
        passages.append(chunk)

print('DONE.\n')

chunked_corpus = {'title': passage_titles, 'text': passages}

print('Processed {:,} passages.'.format(len(chunked_corpus['title'])))


Before splitting, 3 articles.

Splitting...
DONE.

Processed 90 passages.


## Create DPR Embeddings

In [6]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizerFast
import torch
import time
import datetime
import math
import numpy as np

# Model and tokeniser loading
ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-multiset-base")
ctx_tokenizer = DPRContextEncoderTokenizerFast.from_pretrained("facebook/dpr-ctx_encoder-multiset-base")

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Moving the model to the device
ctx_encoder = ctx_encoder.to(device=device)

# Tokenize 
num_passages = len(chunked_corpus['title'])
print('Tokenizing {:,} passages for DPR...'.format(num_passages))

outputs = ctx_tokenizer(
    chunked_corpus['title'],
    chunked_corpus['text'],
    truncation=True,
    padding='longest',
    max_length=512,
    return_tensors='pt'
)

print("DONE")

input_ids = outputs['input_ids']

# Vectorisation
torch.set_grad_enabled(False)
t0 = time.time()
step = 0
batch_size = 16
num_batches = math.ceil(num_passages / batch_size)
embeds_batches = []
print('Processing {:,} batches...'.format(num_passages))
print(input_ids.shape)

for i in range(0, num_passages, batch_size):
    if step % 100 == 0 and not step == 0:
        elapsed = time.time() - t0
        print('  Batch {:>5} of {:>5} Elapsed {:}'.format(step, num_batches, format_time(elapsed)))

    batch_ids = input_ids[i:i + batch_size].to(device=device)

    outputs = ctx_encoder(
        batch_ids,
        return_dict=True
    )

    embeddings = outputs["pooler_output"]
    embeddings = embeddings.detach().cpu().numpy()
    embeds_batches.append(embeddings)
    step += 1

print('DONE.')

embeddings = np.concatenate(embeds_batches, axis=0)
print('Size of dataset embeddings:', embeddings.shape)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-multiset-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected i

Tokenizing 90 passages for DPR...
DONE
Processing 90 batches...
torch.Size([90, 166])
DONE.
Size of dataset embeddings: (90, 768)


## FAISS Index(Facebook Al Similarity Search)


In [7]:
# FAISS Index
import faiss
import time

import datetime

def format_time(elapsed):
    '''
    Verilen süreyi saniye cinsinden alır ve hh:mm:ss biçiminde döndürür.
    '''
    elapsed_rounded = int(round(elapsed))
    return str(datetime.timedelta(seconds=elapsed_rounded))


# FAISS index settings
dim = 768  # Embedding size
m = 128    # HNSW parameter

# Creating a FAISS index
index = faiss.IndexHNSWFlat(dim, m, faiss.METRIC_INNER_PRODUCT)

# Time measurement
t0 = time.time()

# Training and indexing embeddings
index.train(embeddings)
index.add(embeddings)

# Calculating elapsed time
elapsed = time.time() - t0
print('Done.')
print('Adding embeddings to index took:', format_time(elapsed))


Done.
Adding embeddings to index took: 0:00:00


In [8]:
# Question Encoder Installation and Interrogation
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizerFast

# Question encoder and tokeniser installation
q_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-multiset-base")
q_encoder = q_encoder.to(device=device)

q_tokenizer = DPRQuestionEncoderTokenizerFast.from_pretrained("facebook/dpr-question_encoder-multiset-base")

# Create query embedding
input_ids = q_tokenizer.encode("When the United States tested the first nuclear weapon?", return_tensors="pt")
input_ids = input_ids.to(device)

outputs = q_encoder(input_ids)
q_embed = outputs['pooler_output']
q_embed = q_embed.detach().cpu().numpy()

print("Query embedding:", q_embed.shape)

# Search on the FAISS index
k = 3  # Number of nearest neighbours to rotate
D, I = index.search(q_embed, k=k)

print("Closest matching indices:", I)
print("Inner Products:", D)

Some weights of the model checkpoint at facebook/dpr-question_encoder-multiset-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Query embedding: (1, 768)
Closest matching indices: [[4 7 0]]
Inner Products: [[72.76705  72.637665 69.93948 ]]


In [12]:
import textwrap

# Wrap text to fit 80 characters
wrapper = textwrap.TextWrapper(width=80)

# For the nearest 'k' result
retrieved_texts = []
for i in I[0]:
    title = chunked_corpus['title'][i]
    passage = chunked_corpus['text'][i]
    retrieved_texts.append(passage)

    print('Index:', i)
    print('Article Title:', title, '\n')
    print('Passage:')
    print(wrapper.fill(passage))
    print(' ')


Index: 4
Article Title: wiki boiling water reactor History of nuclear power 

Passage:
part of the Manhattan Project, the Allied effort to create atomic bombs during
World War II. It led to the building of larger single-purpose production
reactors, such as the X-10 Pile, for the production of weapons-grade plutonium
for use in the first nuclear weapons. The United States tested the first nuclear
weapon in July 1945, the Trinity test, with the atomic bombings of Hiroshima and
Nagasaki taking place one month later. In August 1945, the first widely
distributed account of nuclear energy, the pocketbook The Atomic Age, was
released. It discussed the peaceful future uses of nuclear energy and depicted
 
Index: 7
Article Title: wiki boiling water reactor History of nuclear power 

Passage:
of U.S. reactor technology and encouraged development by the private sector. The
F-1 (from "First Physical Reactor") was a research reactor operated by the
Kurchatov Institute in Moscow, USSR. When started 

## Ask Questions

In [10]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Model and tokeniser loading
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base')
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')

# Moving the model to the device
t5_model = t5_model.to(device)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
# Merge undeleted texts
context = " ".join(retrieved_texts)

# Question List
questions = [
    "What groundbreaking discovery did John Cockcroft, Ernest Walton, and Ernest Rutherford make in 1932 regarding lithium atoms?",
    "When the United States tested the first nuclear weapon?",
    "Who discovered the neutron in the same year that induced radioactivity was first observed?",
    "What did Otto Hahn, Fritz Strassmann, Lise Meitner, and Otto Robert Frisch discover in their experiments with neutron-bombarded uranium in 1938?",
    "What did Enrico Fermi focus on in the 1930s to enhance induced radioactivity?",
    "Why did scientists worldwide seek government support for nuclear fission research just before World War II?"
]

# Answer retrieval loop
for question in questions:
    # Combining query and context
    input_text = "question: {} context: {}".format(question, context)

    # Processing input text with tokeniser
    input_ids = t5_tokenizer.encode(input_text, return_tensors='pt').to(device)

    # Creating an answer with the model
    outputs = t5_model.generate(input_ids)
    answer = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)

    print("Question:", question)
    print("Generated Answer:", answer)
    print("="*50)  # We add a line to separate questions and answers

Question: What groundbreaking discovery did John Cockcroft, Ernest Walton, and Ernest Rutherford make in 1932 regarding lithium atoms?
Generated Answer: lithium atoms were "split" by protons from a proton accelerator
Question: When the United States tested the first nuclear weapon?
Generated Answer: July 1945
Question: Who discovered the neutron in the same year that induced radioactivity was first observed?
Generated Answer: John Cockcroft, Ernest Walton, and Ernest Rutherford
Question: What did Otto Hahn, Fritz Strassmann, Lise Meitner, and Otto Robert Frisch discover in their experiments with neutron-bombarded uranium in 1938?
Generated Answer: immense amounts of energy were released
Question: What did Enrico Fermi focus on in the 1930s to enhance induced radioactivity?
Generated Answer: a proton accelerator
Question: Why did scientists worldwide seek government support for nuclear fission research just before World War II?
Generated Answer: Manhattan Project


In [18]:
# Merge undeleted texts
context = " ".join(retrieved_texts)

# Question List
questions = [
    "Who were the physicists involved in the first artificial fission of atoms?",
    "What discovery led to the Manhattan Project?",
    "When did John Cockcroft, Ernest Walton, and Ernest Rutherford discover the energy release from splitting lithium atoms?",
    "Who discovered the neutron in 1932?",
    "What did Frédéric and Irène Joliot-Curie discover in 1934?",
    "What did experiments with slow neutrons lead Enrico Fermi to believe he had created?",
    "Who conducted experiments with neutron-bombarded uranium in 1938?",
    "What surprising discovery did Otto Hahn and Fritz Strassmann make in 1938?",
    "Who first recognized the potential of fission reactions for a chain reaction?",
    "When did Frédéric Joliot-Curie announce the experimental confirmation of a self-sustaining nuclear chain reaction?",
    "What was the purpose of the Chicago Pile-1 reactor?",
    "When did Chicago Pile-1 achieve criticality?",
    "What was the goal of the Manhattan Project?",
    "When was the first nuclear weapon tested by the United States?",
    "Who authored the pocketbook 'The Atomic Age' in August 1945?",
    "What did Glenn Seaborg envision for the future of nuclear energy?",
    "What did Eugene Wigner and Alvin Weinberg patent in August 1945?",
    "When was the EBR-I experimental station in Arco, Idaho, first used to generate electricity?",
    "What did Dwight Eisenhower emphasize in his 'Atoms for Peace' speech?",
    "What did the Atomic Energy Act of 1954 enable in the United States?",
    "When did the Obninsk Nuclear Power Plant become the world's first nuclear power plant to generate electricity for a power grid?",
    "Where was the world's first commercial nuclear power station located?",
    "What were the initial capacities of the Calder Hall reactors?",
    "When did the U.S. Army Nuclear Power Program formally commence?",
    "What was the first commercial nuclear station to become operational in the United States?",
    "What impact did the cancellation of a nuclear-powered aircraft carrier contract have on nuclear reactor design adoption?",
    "When was EURATOM launched?",
    "What organization was launched alongside the European Economic Community in 1957?",
    "Where did the first major nuclear reactor accident occur?",
    "What caused the accident at the 3 MW SL-1 reactor?",
    "How many crew fatalities resulted from the accident on the Soviet submarine K-27 in 1968?",
    "How did the global installed nuclear capacity rise from 1960 to the late 1980s?",
    "What was the peak of global nuclear capacity under construction in the late 1970s and early 1980s?",
    "How many nuclear units were cancelled in the United States between 1975 and 1980?",
    "Why was Alvin Weinberg fired from Oak Ridge National Laboratory in 1972?",
    "What was the goal of the test program initiated by Idaho National Laboratory in the late 1970s?",
    "What factors contributed to rising economic costs in nuclear power plant construction during the 1970s and 1980s?",
    "What effect did the 1973 oil crisis have on nuclear power development in France and Japan?",
    "How did the French plan, known as the Messmer plan, aim to reduce dependence on oil?",
    "What role did local opposition play in the cancellation of the proposed Bodega Bay nuclear power station?",
    "When did the first significant anti-nuclear protests emerge in Germany?",
    "What was the significance of the 'Atoms for Peace' speech by Dwight Eisenhower in 1953?",
    "What regulations were implemented in the United States following increased public hostility to nuclear power in the early 1970s?",
    "How did the regulatory changes in the United States affect the license procurement process for nuclear power plants?",
    "How did utility proposals for nuclear generating stations change in the United States between 1974 and 1976?",
    "What was the outcome of the Vermont Yankee Nuclear Power Corp. v. Natural Resources Defense Council, Inc. case?",
    "What was the main consequence of the Three Mile Island accident according to the Nuclear Regulatory Commission?",
    "How did the bankruptcy of Public Service Company of New Hampshire affect the nuclear power industry?",
    "What factors contributed to the shift in electricity generation to coal-fired power plants in the 1980s?",
    "What did President Jimmy Carter call the energy crisis in 1977?",
    "What was the significance of the construction of the first commercial-scale breeder reactor in France?",
    "What was the result of the attack on the Superphenix reactor in France in 1982?",
    "When did the Chernobyl disaster occur?",
    "Where was the Chernobyl Nuclear Power Plant located?",
    "What is considered the worst nuclear disaster in history?",
    "How many personnel were involved in the initial emergency response and decontamination of Chernobyl?",
    "How much did the Chernobyl disaster cost in adjusted inflation terms?",
    "How did the Chernobyl disaster affect the regulation of Western reactors?",
    "What changes were made to the RBMK reactors after the Chernobyl accident?",
    "How many RBMK reactors are still in use today?",
    "What were the primary reasons for the slowdown in global nuclear capacity growth after the late 1980s?",
    "How many nuclear units were cancelled in the United States ultimately?",
    "Who authored the cover story in the 11 February 1985 issue of Forbes magazine criticizing the U.S. nuclear power program?",
    "How did the Three Mile Island accident affect new plant constructions in many countries?",
    "What was the primary cost of implementing regulatory changes after the Three Mile Island accident?",
    "What was the significance of the shutdown of two nuclear power stations in the Tennessee Valley in the 1980s?",
    "What did the anti-nuclear protests in the late 1960s and early 1970s in Europe and North America lead to?",
    "What was the frequency of new nuclear reactor startups globally during the 1980s?",
    "When did utility proposals for nuclear generating stations peak in the United States?",
    "What was the impact of the 1979 oil crisis on countries heavily reliant on oil for electric generation?"
]

In [19]:
# Answer retrieval loop
for question in questions:
    # Combining query and context
    input_text = "question: {} context: {}".format(question, context)

    # Processing input text with tokeniser
    input_ids = t5_tokenizer.encode(input_text, return_tensors='pt').to(device)

    # Creating an answer with the model
    outputs = t5_model.generate(input_ids)
    answer = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)

    print("Question:", question)
    print("Generated Answer:", answer)
    print("="*50)  # We add a line to separate questions and answers

Question: Who were the physicists involved in the first artificial fission of atoms?
Generated Answer: John Cockcroft, Ernest Walton, and Ernest Rutherford
Question: What discovery led to the Manhattan Project?
Generated Answer: artificial fission of atoms
Question: When did John Cockcroft, Ernest Walton, and Ernest Rutherford discover the energy release from splitting lithium atoms?
Generated Answer: 1932
Question: Who discovered the neutron in 1932?
Generated Answer: John Cockcroft, Ernest Walton, and Ernest Rutherford
Question: What did Frédéric and Irène Joliot-Curie discover in 1934?
Generated Answer: lithium atoms were "split" by protons from a proton accelerator
Question: What did experiments with slow neutrons lead Enrico Fermi to believe he had created?
Generated Answer: a self-sustaining nuclear chain reaction
Question: Who conducted experiments with neutron-bombarded uranium in 1938?
Generated Answer: Niels Bohr and Albert Einstein
Question: What surprising discovery did Ott

KeyboardInterrupt: 

In [21]:
import time
from tqdm import tqdm  # We add the tqdm library for the progress bar

total_time_start = time.time()  # Start time for total time calculation
total_questions = len(questions)
progress_bar = tqdm(total=total_questions, desc="Progress", position=0)

# Answer retrieval loop
for question in questions:
    # Combining query and context
    input_text = "question: {} context: {}".format(question, context)

    # Processing input text with tokeniser
    input_ids = t5_tokenizer.encode(input_text, return_tensors='pt').to(device)

    # Creating an answer with the model
    start_time = time.time()
    outputs = t5_model.generate(input_ids)
    end_time = time.time()
    answer = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Printing results
    print("Question:", question)
    print("Generated Answer:", answer)
    print("Time taken:", end_time - start_time, "seconds")  # Print answer generation time
    print("=" * 50)  # We add a line to separate questions and answers

    # Update the progress bar
    progress_bar.update(1)

total_time_end = time.time()  # End time for total time calculation
total_execution_time = total_time_end - total_time_start
progress_bar.close()

print("Total Execution Time:", total_execution_time, "seconds")  # Print total time


Progress:   1%|▏         | 1/70 [00:04<05:12,  4.53s/it]

Question: Who were the physicists involved in the first artificial fission of atoms?
Generated Answer: John Cockcroft, Ernest Walton, and Ernest Rutherford
Time taken: 4.508236885070801 seconds


Progress:   3%|▎         | 2/70 [00:08<04:49,  4.26s/it]

Question: What discovery led to the Manhattan Project?
Generated Answer: artificial fission of atoms
Time taken: 4.053751468658447 seconds


Progress:   4%|▍         | 3/70 [00:11<03:57,  3.55s/it]

...

Question: What was the frequency of new nuclear reactor startups globally during the 1980s?
Generated Answer: a high rate
Time taken: 2.2181081771850586 seconds


Progress:  99%|█████████▊| 69/70 [03:23<00:03,  3.06s/it]

Question: When did utility proposals for nuclear generating stations peak in the United States?
Generated Answer: World War II
Time taken: 2.190495252609253 seconds


Progress: 100%|██████████| 70/70 [03:26<00:00,  2.95s/it]

Question: What was the impact of the 1979 oil crisis on countries heavily reliant on oil for electric generation?
Generated Answer: the private sector
Time taken: 3.127873182296753 seconds
Total Execution Time: 206.37408351898193 seconds
